[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Aplicaciones-Financieras/blob/main/Semana9/2_Manejo_de_portafolios_Markovitz.ipynb) 

# Introducción

Trataremos de encontrar portafolios óptimos, utilizando como medidores la varianza de los retornos, así como el valor esperado de los retornos en un año (el valor que se espera ganar)

# Dependencias

In [ ]:
!pip install yfinance -U plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 7.4 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [ ]:
import yfinance as yf

import pandas as pd
import numpy as np

import cvxopt as opt

import plotly.express as px

# Datos


Los datos con los que trabajaremos son el precio de acciones de Google (GOOG), Apple (AAPL), IBM (IBM), Microsoft (MSFT) y ExxonMobil (XOM) del último año. 

Para obtener los datos, usaremos [```yfinance```](https://github.com/ranaroussi/yfinance) y nos centraremos en el precio de cierre ajustado (Adj Close).

In [ ]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "GOOG AAPL IBM MSFT XOM",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "1y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        # group_by = 'ticker',
    ).loc[:, 'Adj Close']
data

[*********************100%***********************]  5 of 5 completed


,AAPL,GOOG,IBM,MSFT,XOM
Date,,,,,
2021-08-03,146.522873,136.279999,131.182800,284.794495,55.384357
2021-08-04,146.115219,136.028503,129.990005,284.189423,54.090153
2021-08-05,146.224579,136.940002,129.999100,287.175049,54.432739
2021-08-06,145.527512,137.035995,131.201004,287.115570,55.060806
2021-08-09,145.477722,138.001999,130.095764,285.994690,54.432739
...,...,...,...,...,...
2022-07-27,156.789993,113.599998,129.119995,268.739990,91.570000
2022-07-28,157.350006,114.589996,129.220001,276.410004,92.639999
2022-07-29,162.509995,116.639999,130.789993,280.739990,96.930000


In [ ]:
data.shift()

,AAPL,GOOG,IBM,MSFT,XOM
Date,,,,,
2021-08-03,NaN,NaN,NaN,NaN,NaN
2021-08-04,146.522873,136.279999,131.182800,284.794495,55.384357
2021-08-05,146.115219,136.028503,129.990005,284.189423,54.090153
2021-08-06,146.224579,136.940002,129.999100,287.175049,54.432739
2021-08-09,145.527512,137.035995,131.201004,287.115570,55.060806
...,...,...,...,...,...
2022-07-27,151.600006,105.440002,128.080002,251.899994,89.629997
2022-07-28,156.789993,113.599998,129.119995,268.739990,91.570000
2022-07-29,157.350006,114.589996,129.220001,276.410004,92.639999


Obtenemos los retornos logarítmicos anualizados para cada activo.

In [ ]:
annual_returns = np.log(data / data.shift()) / (1 / 252) # Para anualizar los retornos
# annual_returns

mean_returns = annual_returns.mean()
# mean_returns

cov_returns = annual_returns.cov()
cov_returns

,AAPL,GOOG,IBM,MSFT,XOM
AAPL,23.101947,18.413456,5.050901,17.789455,4.542960
GOOG,18.413456,27.242433,3.885731,19.864058,3.908019
IBM,5.050901,3.885731,13.720008,3.547500,5.480513
MSFT,17.789455,19.864058,3.547500,22.844474,2.589256
XOM,4.542960,3.908019,5.480513,2.589256,27.798624


In [ ]:
mean_returns

AAPL    0.088406
GOOG   -0.162629
IBM     0.004789
MSFT   -0.035794
XOM     0.531853
dtype: float64

# Markovitz

https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html

> Si $w$ es el vector de pesos de las acciones con retornos esperados $\mu$, entonces el retorno del portafolio es igual al peso de cada acción multiplicado por su retorno, es decir, $w^T \mu$. El riesgo del portafolio en términos de la matriz de covarianzas $\Sigma$ esta dado por $\sigma^2 = w^T \Sigma w$. 

> La razón de Sharpe es el retorno en exceso del portafolio por unidad de riesgo (volatilidad)

$$
SR = \frac{R_p-R_f}{\sigma}
$$

Con esto en mente, crearemos un cantidad $N$ de portafolios con pesos aleatorios y guardaremos tanto los pesos, como el retorno, la volatilidad y la razon de Sharpe para cada uno.

**NOTA**: Los comentarios en los siguientes 2 bloques de código son para que al quitarlo veamos los portafolios PUROS (todo el capital a una sola acción)

In [ ]:
np.random.seed(1995)

N = 10000
#N = 5

k = annual_returns.shape[1]

weights = np.zeros((N, k))
returns = np.zeros(N)
volatilities = np.zeros(N)
sharpe_ratios = np.zeros(N)

weights2 = np.array([[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]])

weights.shape

(10000, 5)

In [ ]:
for i in range(N):
    w = np.random.random(k)
    w /= np.sum(w)

    #w = weights2[i]
    #weights[i, :] = weights2[i]

    weights[i, :] = w

    returns[i] = np.dot(mean_returns, w)

    volatilities[i] = np.sqrt(np.dot(w.T, np.dot(cov_returns, w))) # w.T @ cov_returns @ w

    sharpe_ratios[i] = returns[i] / volatilities[i]

Graficamos la volatilidad contra el retorno de cada portafolio generado y coloreamos en función de la razón de Sharpe. Los portafolios (casi) óptimos, serían aquellos que tienen el mayor retorno para cierto nivel de volatilidad, o dicho de otra manera, los que tienen la menor volatilidad para algún retorno especificado.

In [ ]:
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

In [ ]:
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

> La optimización del portafolio se puede ver como un problema de optimización convexa y una solución puede encontrarse usando programación cuadrática. Si denotamos el retorno objetivo como $\mu^*$, el problema a resolver para el portafolio sólo con posiciones largar es:

\begin{align}
    \text{min}_w && w^T \Sigma w \\
    \text{s.a.} && w^t \mu \geq \mu^*  \\
    && w^T \mathbf{1} = 1 \\
    && w_i \geq 0
\end{align}

Para resolverlo, ocuparemos la función [```covxopt.solvers.qp```](https://cvxopt.org/userguide/coneprog.html#quadratic-programming). Esta requiere que el problema de optimización se encuentre en la forma general. A saber, la forma general de un problema de programación cuadrática es la siguiente:

\begin{align}
    \text{min}_x && \frac{1}{2}x^TPx + q^Tx \\
    \text{s.a.} && Gx \preceq h \\
    && Ax = b
\end{align}

In [ ]:
mu_star = .35

In [ ]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

In [ ]:
results = opt.solvers.qp(P, q, G, h, A, b)
results

     pcost       dcost       gap    pres   dres
 0:  9.0454e+00  8.3322e+00  1e+01  3e+00  4e+00
 1:  9.0710e+00  8.8038e+00  2e+00  6e-01  7e-01
 2:  1.2105e+01  1.3378e+01  6e+00  5e-01  6e-01
 3:  1.4339e+01  1.4374e+01  1e+00  5e-02  6e-02
 4:  1.4843e+01  1.4824e+01  6e-02  2e-03  2e-03
 5:  1.4839e+01  1.4839e+01  7e-04  2e-05  2e-05
 6:  1.4839e+01  1.4839e+01  7e-06  2e-07  2e-07
 7:  1.4839e+01  1.4839e+01  7e-08  2e-09  2e-09
Optimal solution found.


{'dual infeasibility': 2.0671571353596175e-09,
 'dual objective': 14.839042497481477,
 'dual slack': 6.756398525771023e-09,
 'gap': 6.964466381063939e-08,
 'iterations': 7,
 'primal infeasibility': 1.7181565857674482e-09,
 'primal objective': 14.839042527640604,
 'primal slack': 7.147262007295758e-11,
 'relative gap': 4.6933394673180345e-09,
 's': <6x1 matrix, tc='d'>,
 'status': 'optimal',
 'x': <5x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 'z': <6x1 matrix, tc='d'>}

Los pesos del portafolio óptimo los obtenemos de la llave ```x```

In [ ]:
w = np.asarray(results['x']).reshape((-1))
w

array([0.21861263, 0.        , 0.16109889, 0.00000001, 0.62028846])

In [ ]:
# Impresión bonita de los números
np.set_printoptions(suppress=True)

In [ ]:
w = np.asarray(results['x']).reshape((-1))
w

array([0.21861263, 0.        , 0.16109889, 0.00000001, 0.62028846])

In [ ]:
np.dot(mean_returns, w)

0.349999999370038

La volatilidad de ```primal objective```

In [ ]:
np.sqrt(results['primal objective'])

3.852147781126862

Si variamos los retornos objetivos, podemos obtener la **Frontera Eficiente**, que esta constituida de los portafolios óptimos para distintos niveles del retorno.

In [ ]:
mu_stars = np.linspace(.20, .39, 20)
mu_stars

array([0.2 , 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 ,
       0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39])

In [ ]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

mu_stars = np.linspace(.20, .39, 20)

ws = np.zeros((len(mu_stars), k))
mus = np.zeros(len(mu_stars))
sigmas = np.zeros(len(mu_stars))

for i, mu_star in enumerate(mu_stars):
    try:
        h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
        results = opt.solvers.qp(P, q, G, h, A, b)

        w = np.asarray(results['x']).reshape((-1))
        ws[i, :] = w
        mus[i] = np.dot(mean_returns, w)
        sigmas[i] = np.sqrt(results['primal objective'])
    except:
        print('domain error')

     pcost       dcost       gap    pres   dres
 0:  9.0450e+00  8.0568e+00  1e+01  3e+00  4e+00
 1:  9.0657e+00  8.4497e+00  1e+00  3e-01  3e-01
 2:  9.9547e+00  9.4460e+00  2e+00  1e-01  2e-01
 3:  9.9700e+00  9.9228e+00  6e-02  2e-03  2e-03
 4:  9.9704e+00  9.9699e+00  6e-04  2e-05  2e-05
 5:  9.9704e+00  9.9704e+00  6e-06  2e-07  2e-07
 6:  9.9704e+00  9.9704e+00  6e-08  2e-09  2e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  9.0450e+00  8.0737e+00  1e+01  3e+00  4e+00
 1:  9.0659e+00  8.4668e+00  1e+00  3e-01  4e-01
 2:  1.0065e+01  9.6066e+00  2e+00  2e-01  2e-01
 3:  1.0152e+01  1.0094e+01  9e-02  4e-03  5e-03
 4:  1.0164e+01  1.0163e+01  1e-03  4e-05  6e-05
 5:  1.0164e+01  1.0164e+01  1e-05  4e-07  6e-07
 6:  1.0164e+01  1.0164e+01  1e-07  4e-09  6e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  9.0450e+00  8.0909e+00  1e+01  3e+00  4e+00
 1:  9.0662e+00  8.4849e+00  1e+00  3e-01  4e-01
 2:  1.0180e+01  9.7842e

Finalmente, podemos graficar la frontera eficiente. Se puede observar que, pese a que las simulaciones nos dieron portafolios a la frontera, no eran realmente óptimos.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_traces(
    [
        go.Scatter(
            x = volatilities, y = returns, 
            marker = dict(
                color = sharpe_ratios,
                colorbar = dict(title="Razón de Sharpe")
            ), 
            mode = 'markers', 
            showlegend = False
        ), 
        go.Scatter(
            x = sigmas, y = mus, 
            mode = 'lines + markers',  
            showlegend = False
        )
])

fig.update_layout(
    xaxis_title = 'Volatilidad',
    yaxis_title = 'Retorno'
)



Notas

1. Se están usando los retornos sin considerar la tasa libre de riesgo.

Ejercicios

1. Encontrar y graficar el portafolio óptimo de acuerdo a la razón de Sharpee 
2. Agregar los puntos de los portafolios que contienen únicamente un activo
3. Encontrar el portafolio con la menor volatilidad
4. Restar la tasa libre de riesgo a los retornos

# Mean-Variance Choice

El portafolio óptimo también se podría obtener maximizando, respecto a $w$,

$$
U(\mu, \Sigma; w) = w^T\mu - \frac{\delta}{2}w^T\Sigma w
$$

donde $\delta > 0$ es el parámetro de aversión al riesgo. La condición de primer orden para maximizarla es

$$
\mu = \delta \Sigma w
$$

lo que implica el siguiente diseño para un portafolio con riesgo:

$$
w = \left( \delta \Sigma \right)^{-1} \mu
$$

Es un sistema de ecuaciones lineales que podemos resolver con [```np.linalg.solve```](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

In [ ]:
delta = .2

np.linalg.solve(delta * cov_returns,  mean_returns)

array([ 0.07784051, -0.10280288, -0.04365081,  0.01587235,  0.10452039])

# Ligas interesantes

1. https://python-advanced.quantecon.org/black_litterman.html